# Generate Custom Angular Component

In this notebook, we will use the "Titan Text G1 - Premier" foundational model and the "Titan Embeddings G1 - Text" embeddings model to analyze the code structure of all Angular components in this application. We will then generate a new custom component using the knowledge stored in an in-memory vector database.

## Task 1: Environment Setup

In this task, you set up your environment.

In [ ]:
# Ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime', region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## Task 2: Analyze Angular Components

In this task, we will use the embeddings model to analyze the code structure of all Angular components in this application.

In [ ]:
# Load and analyze Angular component files
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import LocalDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter

# Initialize embeddings model
embeddings_model = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

# Load all files in the Angular project directory
loader = LocalDirectoryLoader(directory="../src/app")
documents = loader.load()
print(f"Loaded {len(documents)} documents.")

# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print(f"Split into {len(docs)} chunks.")

# Create vector store
vectorstore = FAISS.from_documents(docs, embeddings_model)
print("Vector store created.")

## Task 3: Generate Custom Component

In this task, we will use the foundational model to generate a new custom component using the knowledge stored in the in-memory vector database.

In [ ]:
# Generate custom component using foundational model
from langchain_aws import ChatBedrock
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize foundational model
chat_model = ChatBedrock(model_id="amazon.titan-text-premier-v1:0", client=bedrock_client)

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant that generates Angular components based on the analyzed code structure."),
    ("user", "Generate a new custom Angular component that fits well with the existing components.")
])

# Create chain
chain = prompt_template | chat_model | StrOutputParser()

# Generate component
response = chain.invoke({"input": "Generate a new custom Angular component."})
print(response)

## Task 4: Review Generated Component

In this task, we will review the generated component and make any necessary adjustments.

In [ ]:
# Review the generated component
generated_component = response
print("Generated Component:")
print(generated_component)

## Conclusion

In this notebook, we used the "Titan Text G1 - Premier" foundational model and the "Titan Embeddings G1 - Text" embeddings model to analyze the code structure of all Angular components in this application. We then generated a new custom component using the knowledge stored in an in-memory vector database.